In [ ]:
#importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import numpy as np 
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import imageio
import os
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
import pmdarima as pm


In [ ]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('db_mt.xlsx', sheet_name='Sheet1')
print(df.head(10))

        date  value    liter
0 2017-12-31  190.2   301961
1 2018-12-31  192.9   545110
2 2019-12-31  194.8  1029190
3 2020-12-31  197.3  1997490
4 2021-12-31  200.8  2860097
5 2022-12-31  204.7  3209840
6 2023-12-31  208.2  4419242


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    7 non-null      datetime64[ns]
 1   value   7 non-null      float64       
 2   liter   7 non-null      int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 296.0 bytes


In [ ]:
df.drop(['liter'], axis = 1,inplace=True)

In [ ]:
#df.set_index('year',inplace=True)

KeyError: "None of ['year'] are in the columns"

In [ ]:
df=df.sort_index(axis=0)
print(df.head(10))

        date  value
0 2017-12-31  190.2
1 2018-12-31  192.9
2 2019-12-31  194.8
3 2020-12-31  197.3
4 2021-12-31  200.8
5 2022-12-31  204.7
6 2023-12-31  208.2


fh: Optional[int or list or np.array], default = 1

The forecast horizon to be used for forecasting. Default is set to 1 i.e. forecast one point ahead. When integer is passed it means N continuous points in the future without any gap. If you want to forecast values with gaps, you must pass an array e.g. np.arange([13, 25]) will skip the first 12 future points and forecast from the 13th point till the 24th point ahead (note in numpy right value is inclusive and left is exclusive).

In [ ]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=42, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,42
1,Target,value
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(7, 1)"
5,Transformed data shape,"(7, 1)"
6,Transformed train set shape,"(6, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [ ]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Checking Train-Test Split

In [ ]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Comparing Models

The most basic assumption of time series forecasting is that past patterns continue into the future. 
Based on this assumption, in the short-term future forecast, since the environment in which the data is generated is similar to the present, 
the uncertainty is small and the forecast can be well-predicted. However, as we move towards long-term future forecasting, the possibility of the environment in which data is generated is likely to change, and unpredictable situations occur, and as errors continue to accumulate, uncertainty about the forecast data is bound to increase.

We compare the performance of various models and try to select the 4 models with the best performance.

In [ ]:
top4 = exp.compare_models(n_select = 4)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
theta,Theta Forecaster,0.3637,0.3578,0.9067,0.9067,0.0045,0.0045,0.0367
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,0.4052,0.3981,1.0266,1.0266,0.0051,0.0051,0.0500
exp_smooth,Exponential Smoothing,0.4075,0.4005,1.0287,1.0287,0.0051,0.0051,0.0267
polytrend,Polynomial Trend Forecaster,0.4076,0.4006,1.0289,1.0289,0.0051,0.0051,3.8433
ets,ETS,0.4076,0.4006,1.0289,1.0289,0.0051,0.0051,0.0367
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,0.4142,0.4068,1.0513,1.0513,0.0052,0.0052,1.8967
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,0.4269,0.4195,1.0742,1.0742,0.0053,0.0053,0.0633
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,0.4406,0.4326,1.1200,1.1200,0.0055,0.0055,0.0500
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.4406,0.4326,1.1200,1.1200,0.0055,0.0055,0.0667
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,0.4406,0.4326,1.1200,1.1200,0.0055,0.0055,2.1867


## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data. 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

## Difference Plot Using Order

In [ ]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":10})

## ARIMA model

The ARIMA model is a model based on the Autoregression moving average (ARMA), but since the ARMA model is only a model for stationary time series, it is implemented as an ARIMA model including normalizing the abnormal data.

In [ ]:
arima = exp.create_model('arima')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = arima, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,1.3880,1.3674,3.1923,3.1923,0.0162,0.0163
1,2020,1.7305,1.7129,4.0955,4.0955,0.0204,0.0206
2,2021,1.6827,1.6448,4.4591,4.4591,0.0218,0.0220
Mean,NaT,1.6004,1.5750,3.9156,3.9156,0.0195,0.0196
SD,NaT,0.1515,0.1494,0.5326,0.5326,0.0024,0.0024



## Theta Forecaster
The series is tested for seasonality using the test outlined in A&N. 
If deemed seasonal, the series is seasonally adjusted using a classical multiplicative decomposition before applying the theta method. The resulting forecasts are then reseasonalised.

In [ ]:
theta = exp.create_model('theta')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 7},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.0870,0.0857,0.2000,0.2000,0.0010,0.0010
1,2020,0.4986,0.4935,1.1800,1.1800,0.0059,0.0059
2,2021,0.5057,0.4943,1.3400,1.3400,0.0065,0.0066
Mean,NaT,0.3637,0.3578,0.9067,0.9067,0.0045,0.0045
SD,NaT,0.1957,0.1924,0.5039,0.5039,0.0025,0.0025


## ridge_cds_dt

In [ ]:

ets_dt = exp.create_model('ridge_cds_dt')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets_dt, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 7},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## exp_smooth	

In [ ]:

ets_dt = exp.create_model('exp_smooth')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets_dt, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 7},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## polytrend

In [ ]:

ets_dt = exp.create_model('polytrend')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets_dt, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 7},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## ETS
The ETS algorithm is especially useful for datasets with seasonality and other prior assumptions about the data. ETS computes a weighted average over all observations in the input time series dataset as its prediction. The weights are exponentially decreasing over time, rather than the constant weights in simple moving average methods. The weights are dependent on a constant parameter, which is known as the smoothing parameter.

In [ ]:
ets_dt = exp.create_model('ets')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets_dt, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 7},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.0290,0.0286,0.0667,0.0667,0.0003,0.0003
1,2020,0.5070,0.5019,1.2000,1.2000,0.0060,0.0060
2,2021,0.6868,0.6713,1.8200,1.8200,0.0089,0.0089
Mean,NaT,0.4076,0.4006,1.0289,1.0289,0.0051,0.0051
SD,NaT,0.2776,0.2720,0.7259,0.7259,0.0036,0.0036


# AUTO-ARIMA

In [ ]:
df.head()

,date,value
0,2017-12-31,190.2
1,2018-12-31,192.9
2,2019-12-31,194.8
3,2020-12-31,197.3
4,2021-12-31,200.8


In [ ]:
train = df.value[:5]
test = df.value[5:]

In [ ]:
smodel = pm.auto_arima(
    train,
    test='adf',
    start_p=1, start_q=1, start_P=0,
    max_p=3, max_q=3, m=12,
    d=None, D=1,
    seasonal=True,
    information_criterion = 'aic',
    trace=True,
    error_action='ignore', suppress_warnings=True,
)

smodel.summary()

ValueError: Encountered exception in stationarity test ('adf'). This can occur in seasonal settings when a large enough `m` coupled with a large enough `D` difference the training array into too few samples for OLS (input contains 5 samples). Try fitting on a larger training size (raised from LinAlgError: Last 2 dimensions of the array must be square)

In [ ]:
#fit the auto SARIMA model (0,1,2)(0,1,1)[12] on the training set
auto_sarima = sm.tsa.statespace.SARIMAX(train,
                                               order=(0, 1, 2),
                                               seasonal_order=(0, 1, 1, 12),
                                               enforce_stationarity=False,
                                               enforce_invertibility=False)
results_auto_sarima = auto_sarima.fit()

#fit the plot based SARIMA model (1,1,1)(0,1,1)[12] on the training set
plot_sarima = sm.tsa.statespace.SARIMAX(train,
                                               order=(1, 1, 1),
                                               seasonal_order=(0, 1, 1, 12),
                                               enforce_stationarity=False,
                                               enforce_invertibility=False)
results_plot_sarima = plot_sarima.fit()

#forecast on the test set
forecast_auto = results_auto_sarima.get_forecast(steps=len(test))
forecast_plot = results_plot_sarima.get_forecast(steps=len(test))

#get the forecasted values
forecast_auto_values = forecast_auto.predicted_mean
forecast_plot_values = forecast_plot.predicted_mean

#calculate the error metrics for both models
mse_auto = mean_squared_error(test, forecast_auto_values)
rmse_auto = sqrt(mse_auto)

mse_plot = mean_squared_error(test, forecast_plot_values)
rmse_plot = sqrt(mse_plot)

#print the results
print(f"SARIMA(0,1,2)(0,1,1)[12] - MSE: {mse_auto}, RMSE: {rmse_auto}")
print(f"SARIMA(1,1,1)(0,1,1)[12] - MSE: {mse_plot}, RMSE: {rmse_plot}")

#decide which model is better based on lower MSE and RMSE
better_model = 'SARIMA(0,1,2)(0,1,1)[12] auto_sarima' if mse_auto < mse_plot else 'SARIMA(1,1,1)(0,1,1)[12] plot sarima'
print(f"The better model based on MSE is: {better_model}")
better_model = 'SARIMA(0,1,2)(0,1,1)[12] auto_sarima' if rmse_auto < rmse_plot else 'SARIMA(1,1,1)(0,1,1)[12] plot sarima'
print(f"The better model based on RMSE is: {better_model}")


## IMPORT Pycaret


In [ ]:
from pycaret.regression import *


Setting up the model is extremely easy

In [ ]:

reg = setup(data = train, 
             target = 'value',
             numeric_imputation = 'mean',
             categorical_features = ['year']  , 
            transformation = True, transform_target = True, 
                  combine_rare_levels = True, rare_level_threshold = 0.1,
                  remove_multicollinearity = True, multicollinearity_threshold = 0.95, 
             silent = True)

NameError: name 'train' is not defined

As a data scientist, I can't emphasize more on the usefulness of the function below - instead of pulling every single model, we just need one line to compare 20 different models! This is insane!

In [ ]:

# returns best models - takes a little time to run
top3 = compare_models(n_select = 3)

## Creating baseline model

In [ ]:
#we create a model using light gbm
lightgbm = create_model('lightgbm')

In [ ]:
tuned_lightgbm = tune_model(lightgbm) 

In [ ]:
plot_model(lightgbm, plot = 'error')

In [ ]:

plot_model(tuned_lightgbm, plot='feature') # looks like COVID-19 has played a huge role in sales

In [ ]:
predict_model(tuned_lightgbm);

In [ ]:
final_lightgbm = finalize_model(tuned_lightgbm)